In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [5]:
data = pd.read_excel("../data/fpl_data/full_df.xlsx")

In [6]:
data

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,GW,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,season
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,55,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,55,False,0,1,0.00,0.00,0.00,0.00,0,2020/21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93194,Kyle Walker,DEF,Man City,3.7,0,0,16,0,36.5,369,...,55,False,0,23,0.12,0.14,0.02,1.02,1,2023/24
93195,Jacob Brown,FWD,Luton,-0.7,0,0,1,0,0.5,631,...,49,False,0,23,0.00,0.05,0.05,0.19,0,2023/24
93196,Vicente Guaita,GK,Crystal Palace,0.0,0,0,0,0,0.0,227,...,44,False,0,23,0.00,0.00,0.00,0.00,0,2023/24
93197,Braian Ojeda Rodríguez,MID,Nott'm Forest,0.0,0,0,0,0,0.0,459,...,45,False,0,23,0.00,0.00,0.00,0.00,0,2023/24


In [7]:
data.info(45)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93199 entries, 0 to 93198
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        93199 non-null  object 
 1   position                    93199 non-null  object 
 2   team                        93199 non-null  object 
 3   xP                          93199 non-null  float64
 4   assists                     93199 non-null  int64  
 5   bonus                       93199 non-null  int64  
 6   bps                         93199 non-null  int64  
 7   clean_sheets                93199 non-null  int64  
 8   creativity                  93199 non-null  float64
 9   element                     93199 non-null  int64  
 10  fixture                     93199 non-null  int64  
 11  goals_conceded              93199 non-null  int64  
 12  goals_scored                93199 non-null  int64  
 13  ict_index                   931

In [23]:
data[(data["season"] == "2023/24") & (data["name"] == "Mohamed Salah")][["team", "kickoff_time", "opponent_team", "expected_goal_involvements", "expected_goals", "expected_goals_conceded", "expected_assists"]].tail(14)

,team,kickoff_time,opponent_team,expected_goal_involvements,expected_goals,expected_goals_conceded,expected_assists
83263,Liverpool,2023-10-29T14:00:00Z,16,0.71,0.58,0.45,0.13
83996,Liverpool,2023-11-05T16:30:00Z,12,0.68,0.59,0.81,0.09
84738,Liverpool,2023-11-12T14:00:00Z,4,1.15,0.94,1.57,0.21
85485,Liverpool,2023-11-25T12:30:00Z,13,0.21,0.00,1.35,0.21
86238,Liverpool,2023-12-03T14:00:00Z,10,1.03,0.77,1.22,0.26
86993,Liverpool,2023-12-06T19:30:00Z,17,0.44,0.36,0.25,0.08
87751,Liverpool,2023-12-09T12:30:00Z,8,0.51,0.33,1.74,0.18
88452,Liverpool,2023-12-17T16:30:00Z,14,0.75,0.42,0.63,0.33
89146,Liverpool,2023-12-23T17:30:00Z,1,0.35,0.22,0.80,0.13
89907,Liverpool,2023-12-26T17:30:00Z,6,0.42,0.27,0.76,0.15


In [19]:
prem_list = sorted(data[data["season"]=="2023/24"]["team"].unique())

In [20]:
prem_list.remove("Liverpool")

In [21]:
prem_list

['Arsenal',
 'Aston Villa',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Luton',
 'Man City',
 'Man Utd',
 'Newcastle',
 "Nott'm Forest",
 'Sheffield Utd',
 'Spurs',
 'West Ham',
 'Wolves']

In [47]:
data = data[data["position"] != "GKP"].reset_index(drop=True)
data_use = data[data["season"].isin(["2022/23", "2023/24"])].reset_index(drop=True)

In [48]:
data_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43387 entries, 0 to 43386
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        43387 non-null  object 
 1   position                    43387 non-null  object 
 2   team                        43387 non-null  object 
 3   xP                          43387 non-null  float64
 4   assists                     43387 non-null  int64  
 5   bonus                       43387 non-null  int64  
 6   bps                         43387 non-null  int64  
 7   clean_sheets                43387 non-null  int64  
 8   creativity                  43387 non-null  float64
 9   element                     43387 non-null  int64  
 10  fixture                     43387 non-null  int64  
 11  goals_conceded              43387 non-null  int64  
 12  goals_scored                43387 non-null  int64  
 13  ict_index                   433

In [49]:
data_use = data_use[["name", "position", "team", "opponent_team", "minutes", "goals_scored", "goals_conceded", "clean_sheets", "expected_goal_involvements", "expected_goals", "expected_goals_conceded", "expected_assists", "season", "own_goals", "GW"]]

In [50]:
data_use

,name,position,team,opponent_team,minutes,goals_scored,goals_conceded,clean_sheets,expected_goal_involvements,expected_goals,expected_goals_conceded,expected_assists,season,own_goals,GW
0,Nathan Redmond,MID,Southampton,18,1,0,0,0,0.00,0.00,0.00,0.00,2022/23,0,1
1,Junior Stanislas,MID,Bournemouth,2,1,0,0,0,0.00,0.00,0.00,0.00,2022/23,0,1
2,Armando Broja,FWD,Chelsea,8,15,0,0,0,0.00,0.00,0.00,0.00,2022/23,0,1
3,Fabian Schär,DEF,Newcastle,16,90,1,0,1,0.00,0.00,0.00,0.00,2022/23,0,1
4,Jonny Evans,DEF,Leicester,4,90,0,2,0,0.00,0.00,0.00,0.00,2022/23,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43382,Kyle Walker,DEF,Man City,4,90,0,1,0,0.14,0.02,1.02,0.12,2023/24,0,23
43383,Jacob Brown,FWD,Luton,15,11,0,0,0,0.05,0.05,0.19,0.00,2023/24,0,23
43384,Vicente Guaita,GK,Crystal Palace,5,0,0,0,0,0.00,0.00,0.00,0.00,2023/24,0,23
43385,Braian Ojeda Rodríguez,MID,Nott'm Forest,3,0,0,0,0,0.00,0.00,0.00,0.00,2023/24,0,23


In [40]:
df2022_23 = data_use[data_use["season"] ==  "2022/23"]

In [41]:
df2022_23

,name,position,team,opponent_team,minutes,goals_scored,goals_conceded,clean_sheets,expected_goal_involvements,expected_goals,expected_goals_conceded,expected_assists,season
0,Nathan Redmond,MID,Southampton,18,1,0,0,0,0.00,0.0,0.0,0.00,2022/23
1,Junior Stanislas,MID,Bournemouth,2,1,0,0,0,0.00,0.0,0.0,0.00,2022/23
2,Armando Broja,FWD,Chelsea,8,15,0,0,0,0.00,0.0,0.0,0.00,2022/23
3,Fabian Schär,DEF,Newcastle,16,90,1,0,1,0.00,0.0,0.0,0.00,2022/23
4,Jonny Evans,DEF,Leicester,4,90,0,2,0,0.00,0.0,0.0,0.00,2022/23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26500,Oliver Skipp,MID,Spurs,11,90,0,1,0,0.01,0.0,1.5,0.01,2022/23
26501,Ryan Sessegnon,DEF,Spurs,11,0,0,0,0,0.00,0.0,0.0,0.00,2022/23
26502,Ashley Young,DEF,Aston Villa,5,0,0,0,0,0.00,0.0,0.0,0.00,2022/23
26503,Jeremy Sarmiento Morante,MID,Brighton,2,0,0,0,0,0.00,0.0,0.0,0.00,2022/23


In [54]:
pl2021_22 = pd.read_excel("../data/premier_league_fixtures/pl2021_22.xlsx")
pl2022_23 = pd.read_excel("../data/premier_league_fixtures/pl2022_23.xlsx")
pl2023_24 = pd.read_excel("../data/premier_league_fixtures/pl2023_24.xlsx")

In [59]:
pl2023_24_new = pl2023_24.dropna(subset=["Home Goal", "Away Goal"]).reset_index(drop=True)